In [1]:
# Cell 1: Import necessary libraries and set device
import re
import torch
import torchaudio
from datasets import load_dataset, Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from torch.utils.data import DataLoader
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cuda


In [4]:
import tempfile
import zipfile
import os
from datasets import load_from_disk

zip_path = "small_validation_set.zip"
with tempfile.TemporaryDirectory() as tmpdirname:
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(tmpdirname)
    
    # List the contents to determine the correct path
    extracted_items = os.listdir(tmpdirname)
    print("Extracted items:", extracted_items)
    
    # If you see a folder like "small_validation_set", use that folder
    dataset_path = os.path.join(tmpdirname, "small_validation_set")
    small_validation_set = load_from_disk(dataset_path)
    # Now use small_validation_set as needed


Extracted items: ['small_validation_set']


In [5]:
small_validation_set

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 100
})

In [6]:
# Cell 7: Define data collator without moving tensors to device
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Process audio
        input_features = [self.processor(feature["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features[0] for feature in features]
        # Process labels
        labels = [self.processor.tokenizer(feature["text"]).input_ids for feature in features]

        # Pad inputs and labels
        input_features = torch.stack(input_features)
        labels = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(l) for l in labels],
            batch_first=True,
            padding_value=self.processor.tokenizer.pad_token_id
        )

        # Replace padding token id's of the labels by -100 so they are ignored in the loss computation
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        batch = {
            "input_features": input_features,  # Do not move to device here
            "labels": labels,                  # Do not move to device here
        }
        return batch


In [7]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

c:\Users\srinivas\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\srinivas\.cache\huggingface\hub\models--openai--whisper-tiny. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [8]:
# Cell 8: Create DataLoaders with pin_memory=True
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
# train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=data_collator, pin_memory=True)
# val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=data_collator, pin_memory=True)


In [9]:
import evaluate
import numpy as np

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Convert to numpy arrays and move to CPU
    if isinstance(pred_ids, torch.Tensor):
        pred_ids = pred_ids.cpu().numpy()
    else:
        pred_ids = np.array(pred_ids)

    if isinstance(label_ids, torch.Tensor):
        label_ids = label_ids.cpu().numpy()
    else:
        label_ids = np.array(label_ids)

    # Replace -100 with the pad token ID
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER and CER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}



In [11]:
# Import necessary libraries
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm.auto import tqdm
small_val_dataset = small_validation_set
# Step 1: Set device and clear GPU cache
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Load the fine-tuned Whisper model on GPU
model1 = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-best-model/checkpoint-22832").to(device) #change

# Step 3: Define evaluation training arguments
eval_training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-best-model",
    per_device_eval_batch_size=1,      # Keep batch size low to manage GPU memory
    dataloader_num_workers=0,          # Set to 0 to simplify I/O operations
    remove_unused_columns=False,
    predict_with_generate=True,
    fp16=False,                        # Disable fp16 for stability during evaluation
    evaluation_strategy="no",
    disable_tqdm=False,
    logging_dir="./logs-eval",
    logging_steps=10,
    report_to="none"
)

# Step 4: Initialize Trainer
trainer1 = Seq2SeqTrainer(
    model=model1,
    args=eval_training_args,
    eval_dataset=small_val_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 5: Perform Evaluation with Debug Print Statements
torch.cuda.empty_cache()  # Clear GPU cache
print("Starting evaluation...")

try:
    eval_results = trainer1.evaluate()
    print(f"Final WER: {eval_results['eval_wer']}")
    print(f"Final CER: {eval_results['eval_cer']}")
except RuntimeError as e:
    print(f"Runtime error during evaluation: {e}")
    torch.cuda.empty_cache()


Using device: cuda


c:\Users\srinivas\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\srinivas\AppData\Local\Temp\ipykernel_14916\1373869020.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer1 = Seq2SeqTrainer(


Starting evaluation...


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/100 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Final WER: 0.20552519732847602
Final CER: 0.057348686822848716


In [13]:
import mlflow
import mlflow.pytorch
from mlflow.models.signature import infer_signature

# Step 1: Set MLflow Tracking URI and Experiment Name
mlflow.set_tracking_uri("http://127.0.0.1:5555")  
experiment_name = "Whisper Experiment"
mlflow.set_experiment(experiment_name)

2025/02/16 22:22:42 INFO mlflow.tracking.fluent: Experiment with name 'Whisper Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/459095338461693691', creation_time=1739762562906, experiment_id='459095338461693691', last_update_time=1739762562906, lifecycle_stage='active', name='Whisper Experiment', tags={}>

In [14]:
def log_model_to_mlflow(model, processor, eval_results, eval_training_args, experiment_name):
    with mlflow.start_run(run_name="Whisper-Ft7 Evaluation Run"):
        # Log model parameters
        mlflow.log_param("model_name", "WhisperFineTunedModel7")
        mlflow.log_param("checkpoint", "./whisper-finetuned-dev-clean7/checkpoint-22832")

        # Log evaluation metrics
        mlflow.log_metric("eval_wer", eval_results["eval_wer"])
        mlflow.log_metric("eval_cer", eval_results["eval_cer"])  # Log CER

        # Save the model locally before logging to MLflow
        model_dir = "./whisper_model_mlflow"
        model.save_pretrained(model_dir)
        processor.save_pretrained(model_dir)

        # Log the model to MLflow
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="whisper_model",
            registered_model_name="WhisperFineTunedModel7"
        )
        print("Model and training arguments logged successfully to MLflow.")

log_model_to_mlflow(model1, processor, eval_results, eval_training_args, experiment_name)


2025/02/16 22:22:47 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/02/16 22:23:05 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.20.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.20.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/02/16 22:23:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Whisper

Model and training arguments logged successfully to MLflow.
🏃 View run Whisper-Ft7 Evaluation Run at: http://127.0.0.1:5555/#/experiments/459095338461693691/runs/c2b9f2818f2e4d0c994847c66b520d34
🧪 View experiment at: http://127.0.0.1:5555/#/experiments/459095338461693691


Created version '1' of model 'WhisperFineTunedModel7'.


In [ ]:
small_val_dataset = small_validation_set
# Step 1: Set device and clear GPU cache
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Load the fine-tuned Whisper model on GPU
model2 = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-dev-clean4/checkpoint-8562").to(device) #change

# Step 3: Define evaluation training arguments
eval_training_args2 = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-dev-clean-eval4",
    per_device_eval_batch_size=1,      # Keep batch size low to manage GPU memory
    dataloader_num_workers=0,          # Set to 0 to simplify I/O operations
    remove_unused_columns=False,
    predict_with_generate=True,
    fp16=False,                        # Disable fp16 for stability during evaluation
    evaluation_strategy="no",
    disable_tqdm=False,
    logging_dir="./logs-eval",
    logging_steps=10,
    report_to="none"
)

# Step 4: Initialize Trainer
trainer2 = Seq2SeqTrainer(    #change
    model=model2,  #change
    args=eval_training_args2,
    eval_dataset=small_val_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 5: Perform Evaluation with Debug Print Statements
torch.cuda.empty_cache()  # Clear GPU cache
print("Starting evaluation...")

try:
    eval_results2 = trainer2.evaluate()   #change
    print(f"Final WER: {eval_results2['eval_wer']}")   #change
    print(f"Final CER: {eval_results2['eval_cer']}")
except RuntimeError as e:
    print(f"Runtime error during evaluation: {e}")
    torch.cuda.empty_cache()


In [ ]:
def log_model_to_mlflow2(model, processor, eval_results2, eval_training_args2, experiment_name):
    with mlflow.start_run(run_name="Whisper-Ft4 Evaluation Run"):
        # Log model parameters
        mlflow.log_param("model_name", "WhisperFineTunedModel4")
        mlflow.log_param("checkpoint", "./whisper-finetuned-dev-clean4/checkpoint-8562")

        # Log evaluation metrics
        mlflow.log_metric("eval_wer", eval_results2["eval_wer"])
        mlflow.log_metric("eval_cer", eval_results2["eval_cer"])  # Log CER


        # Save the model locally before logging to MLflow
        model_dir = "./whisper_model_mlflow2"
        model.save_pretrained(model_dir)
        processor.save_pretrained(model_dir)

        # Log the model to MLflow
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="whisper_model4",
            registered_model_name="WhisperFineTunedModel4"
        )
        print("Model and training arguments logged successfully to MLflow.")

log_model_to_mlflow2(model2, processor, eval_results2, eval_training_args2, experiment_name)


2024/12/08 19:31:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'WhisperFineTunedModel4'.
2024/12/08 19:31:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WhisperFineTunedModel4, version 1


Model and training arguments logged successfully to MLflow.
🏃 View run Whisper-Ft4 Evaluation Run at: http://127.0.0.1:5555/#/experiments/1/runs/86f007253e414a6d82bbf66468bfe896
🧪 View experiment at: http://127.0.0.1:5555/#/experiments/1


Created version '1' of model 'WhisperFineTunedModel4'.


In [ ]:


small_val_dataset = small_validation_set


# Step 1: Set device and clear GPU cache
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Load the fine-tuned Whisper model on GPU
model3 = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-dev-clean2/checkpoint-8562").to(device) #change

# Step 3: Define evaluation training arguments
eval_training_args3 = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-dev-clean-eval2",
    per_device_eval_batch_size=1,      # Keep batch size low to manage GPU memory
    dataloader_num_workers=0,          # Set to 0 to simplify I/O operations
    remove_unused_columns=False,
    predict_with_generate=True,
    fp16=False,                        # Disable fp16 for stability during evaluation
    evaluation_strategy="no",
    disable_tqdm=False,
    logging_dir="./logs-eval",
    logging_steps=10,
    report_to="none"
)

# Step 4: Initialize Trainer
trainer3 = Seq2SeqTrainer(    #change
    model=model3,  #change
    args=eval_training_args3,
    eval_dataset=small_val_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 5: Perform Evaluation with Debug Print Statements
torch.cuda.empty_cache()  # Clear GPU cache
print("Starting evaluation...")

try:
    eval_results3 = trainer3.evaluate()   #change
    print(f"Final WER: {eval_results3['eval_wer']}")   #change
    print(f"Final CER: {eval_results3['eval_cer']}")
except RuntimeError as e:
    print(f"Runtime error during evaluation: {e}")
    torch.cuda.empty_cache()


In [ ]:
def log_model_to_mlflow3(model, processor, eval_results3, eval_training_args3, experiment_name):
    with mlflow.start_run(run_name="Whisper-Ft2 Evaluation Run"):
        # Log model parameters
        mlflow.log_param("model_name", "WhisperFineTunedModel2")
        mlflow.log_param("checkpoint", "./whisper-finetuned-dev-clean2/checkpoint-8562")

        # Log evaluation metrics
        mlflow.log_metric("eval_wer", eval_results3["eval_wer"])
        mlflow.log_metric("eval_cer", eval_results3["eval_cer"])  # Log CER
        # Save the model locally before logging to MLflow
        model_dir = "./whisper_model_mlflow3"
        model.save_pretrained(model_dir)
        processor.save_pretrained(model_dir)

        # Log the model to MLflow
        mlflow.pytorch.log_model(
            pytorch_model=model,
            artifact_path="whisper_model2",
            registered_model_name="WhisperFineTunedModel2"
        )
        print("Model and training arguments logged successfully to MLflow.")

log_model_to_mlflow3(model3, processor, eval_results3, eval_training_args3, experiment_name)

2024/12/08 19:32:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'WhisperFineTunedModel2'.
2024/12/08 19:32:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WhisperFineTunedModel2, version 1


Model and training arguments logged successfully to MLflow.
🏃 View run Whisper-Ft2 Evaluation Run at: http://127.0.0.1:5555/#/experiments/1/runs/880374dfa9e5450c8c7e4c802ab3edb4
🧪 View experiment at: http://127.0.0.1:5555/#/experiments/1


Created version '1' of model 'WhisperFineTunedModel2'.


In [18]:
from mlflow.tracking.client import MlflowClient

# Initialize MlflowClient
client = MlflowClient()

# Retrieve all registered model names using the Model Registry API
registered_models = client.search_registered_models()
for model in registered_models:
    print(model.name)  # Print the name of each registered model


WhisperFineTunedModel7


In [19]:
from mlflow.tracking import MlflowClient
import mlflow.pytorch

# Initialize MlflowClient
client = MlflowClient()

# Retrieve all runs for the experiment
experiment = client.get_experiment_by_name(experiment_name)
if not experiment:
    raise ValueError(f"Experiment '{experiment_name}' not found.")

# Dictionary to store model names and their WER metrics
model_metrics = {}

# Iterate through runs in the experiment
for run in client.search_runs(experiment.experiment_id):
    model_name = run.data.params.get("model_name")
    eval_wer = run.data.metrics.get("eval_wer")

    print(model_name)
    if model_name and eval_wer is not None:
        model_metrics[model_name] = eval_wer

# Identify the best model with the lowest WER
best_model_name = min(model_metrics, key=model_metrics.get)
print(f"Best model is: {best_model_name} with WER: {model_metrics[best_model_name]}")


WhisperFineTunedModel7
Best model is: WhisperFineTunedModel7 with WER: 0.20552519732847602


In [20]:
model_metrics

{'WhisperFineTunedModel7': 0.20552519732847602}

In [21]:

try:
    # Retrieve all registered versions of the best model
    registered_versions = client.get_registered_model(name=best_model_name).latest_versions
    # print(registered_versions)
    # Find the latest version with stage "None" (or any other desired stage, e.g., "Production")
    best_model_version = next((v.version for v in registered_versions if v.current_stage == "None"), None)

    if best_model_version is None:
        raise ValueError(f"No version of {best_model_name} is available in stage 'None'.")

    # Load the best model from the Model Registry
    model_uri = f"models:/{best_model_name}/{best_model_version}"
    best_model = mlflow.pytorch.load_model(model_uri)

    # Make predictions using the best model
    print(f"Loaded the best model '{best_model_name}' successfully.")
except mlflow.exceptions.RestException as e:
    print(f"Error: {e}")
    print(f"The model '{best_model_name}' is not registered. Please verify the `registered_model_name`.")


Loaded the best model 'WhisperFineTunedModel7' successfully.


In [ ]:
import soundfile as sf  # Make sure this is imported

def transcribe_audio(audio_file_path, processor, model):
    # Load the audio file
    audio, sample_rate = sf.read(audio_file_path)

    # Ensure the audio is a single channel (mono)
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)

    # Convert the audio to a tensor and ensure it is float32
    audio = torch.tensor(audio, dtype=torch.float32)

    # Resample the audio to 16kHz if necessary
    if sample_rate != 16000:
        resample_transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        audio = resample_transform(audio)

    # Truncate or pad the audio to fit Whisper's requirements
    max_audio_length = 30 * 16000  # Whisper expects a maximum of 30 seconds
    if audio.shape[0] > max_audio_length:
        audio = audio[:max_audio_length]
    else:
        audio = torch.nn.functional.pad(audio, (0, max_audio_length - audio.shape[0]))

    # Process the audio input
    inputs = processor(audio.numpy(), sampling_rate=16000, return_tensors="pt", padding=True)

    # Move inputs to the same device as the model
    input_features = inputs.input_features.to(next(model.parameters()).device)

    # Generate predictions using the model
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # Decode the predictions to text
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

# Example usage
audio_file_path = "inp9.wav"  # Replace with your audio file path

# Transcribe the audio file
transcription = transcribe_audio(audio_file_path, processor, best_model)
print(f"Transcription: {transcription}")


In [24]:
# Example usage2
audio_file_path = "Inputs/fdp1.wav"  # Replace with your audio file path

# Transcribe the audio file
transcription = transcribe_audio(audio_file_path, processor, best_model)
print(f"Transcription: {transcription}")

Transcription:  The quick brown fox jumps over the big wide white penance in lazy land
